In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('data/cleaned_dataset.csv')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,is_null_customer,transaction_value
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-01-12 07:45:00,6.95,13085.0,United Kingdom,False,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-01-12 07:45:00,6.75,13085.0,United Kingdom,False,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-01-12 07:45:00,6.75,13085.0,United Kingdom,False,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-01-12 07:45:00,2.10,13085.0,United Kingdom,False,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-01-12 07:45:00,1.25,13085.0,United Kingdom,False,30.0


In [3]:
clv_df = pd.read_csv('data/clean_clv_dataset.csv')
clv_df.head()

,Customer ID,recency,frequency,monetary,R,F,M,RFM_score
0,12346,326,48,-64.68,2,2,1,221
1,12347,40,253,5633.32,3,4,4,344
2,12348,76,51,2019.40,3,2,3,323
3,12349,19,180,4404.54,4,4,4,444
4,12350,311,17,334.40,2,1,2,212


In [4]:
# Ensure InvoiceDate is datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Get latest date in dataset
latest_date = df['InvoiceDate'].max()

# Get first purchase date per customer
first_purchase = df.groupby('Customer ID')['InvoiceDate'].min()

# Calculate tenure in days as a Series
tenure_days = (latest_date - first_purchase).astype('timedelta64[D]')

# Map to clv_df
clv_df['tenure_days'] = clv_df['Customer ID'].map(tenure_days)

In [5]:
clv_df['avg_order_value'] = clv_df['monetary'] / clv_df['frequency']

In [6]:
clv_df.head()

,Customer ID,recency,frequency,monetary,R,F,M,RFM_score,tenure_days,avg_order_value
0,12346,326,48,-64.68,2,2,1,221,726.0,-1.347500
1,12347,40,253,5633.32,3,4,4,344,516.0,22.266087
2,12348,76,51,2019.40,3,2,3,323,439.0,39.596078
3,12349,19,180,4404.54,4,4,4,444,972.0,24.469667
4,12350,311,17,334.40,2,1,2,212,311.0,19.670588


In [7]:
clv_df['tenure_days'] = clv_df['Customer ID'].map(tenure_days)
clv_df['purchase_velocity'] = clv_df['frequency'] / (clv_df['tenure_days'] / 30)

In [8]:
clv_df.head()

,Customer ID,recency,frequency,monetary,R,F,M,RFM_score,tenure_days,avg_order_value,purchase_velocity
0,12346,326,48,-64.68,2,2,1,221,726.0,-1.347500,1.983471
1,12347,40,253,5633.32,3,4,4,344,516.0,22.266087,14.709302
2,12348,76,51,2019.40,3,2,3,323,439.0,39.596078,3.485194
3,12349,19,180,4404.54,4,4,4,444,972.0,24.469667,5.555556
4,12350,311,17,334.40,2,1,2,212,311.0,19.670588,1.639871


In [9]:
clv_df['avg_days_between_orders'] = clv_df['tenure_days'] / clv_df['frequency']
clv_df.head()

,Customer ID,recency,frequency,monetary,R,F,M,RFM_score,tenure_days,avg_order_value,purchase_velocity,avg_days_between_orders
0,12346,326,48,-64.68,2,2,1,221,726.0,-1.347500,1.983471,15.125000
1,12347,40,253,5633.32,3,4,4,344,516.0,22.266087,14.709302,2.039526
2,12348,76,51,2019.40,3,2,3,323,439.0,39.596078,3.485194,8.607843
3,12349,19,180,4404.54,4,4,4,444,972.0,24.469667,5.555556,5.400000
4,12350,311,17,334.40,2,1,2,212,311.0,19.670588,1.639871,18.294118


In [10]:
clv_df['churn_flag'] = clv_df['recency'] > (clv_df['avg_days_between_orders'] * 1.5)

In [11]:
df['discount_flag'] = df['Description'].str.contains('DISCOUNT|PROMO|SALE|CREDIT', case=False, na=False)

In [12]:
df['discount_flag'].value_counts()

False    1067162
True         209
Name: discount_flag, dtype: int64

In [13]:
discount_ratio = df.groupby('Customer ID')['discount_flag'].mean()
clv_df['discount_use_ratio'] = clv_df['Customer ID'].map(discount_ratio)
clv_df.head()

,Customer ID,recency,frequency,monetary,R,F,M,RFM_score,tenure_days,avg_order_value,purchase_velocity,avg_days_between_orders,churn_flag,discount_use_ratio
0,12346,326,48,-64.68,2,2,1,221,726.0,-1.347500,1.983471,15.125000,True,0.020833
1,12347,40,253,5633.32,3,4,4,344,516.0,22.266087,14.709302,2.039526,True,0.000000
2,12348,76,51,2019.40,3,2,3,323,439.0,39.596078,3.485194,8.607843,True,0.000000
3,12349,19,180,4404.54,4,4,4,444,972.0,24.469667,5.555556,5.400000,True,0.000000
4,12350,311,17,334.40,2,1,2,212,311.0,19.670588,1.639871,18.294118,True,0.000000


In [14]:
cohort_month = df.groupby('Customer ID')['InvoiceDate'].min().dt.to_period('M')
clv_df['cohort_month'] = clv_df['Customer ID'].map(cohort_month)

In [15]:
clv_df.head()

,Customer ID,recency,frequency,monetary,R,F,M,RFM_score,tenure_days,avg_order_value,purchase_velocity,avg_days_between_orders,churn_flag,discount_use_ratio,cohort_month
0,12346,326,48,-64.68,2,2,1,221,726.0,-1.347500,1.983471,15.125000,True,0.020833,2009-12
1,12347,40,253,5633.32,3,4,4,344,516.0,22.266087,14.709302,2.039526,True,0.000000,2010-07
2,12348,76,51,2019.40,3,2,3,323,439.0,39.596078,3.485194,8.607843,True,0.000000,2010-09
3,12349,19,180,4404.54,4,4,4,444,972.0,24.469667,5.555556,5.400000,True,0.000000,2009-04
4,12350,311,17,334.40,2,1,2,212,311.0,19.670588,1.639871,18.294118,True,0.000000,2011-02


In [16]:
clv_df['churn_flag'].value_counts()

True     5371
False     571
Name: churn_flag, dtype: int64

In [17]:
clv_df.to_csv('data/feature_engineered_clv_dataset.csv', index=False)